# Initial calculation code

In [ ]:
import numpy as np

def geom_avg(vals):
    """
    Compute the geometric average of a list of values.
    
    The values need not be a list, but simply anything with a len() and []
    """
    rval=1.0
    count = 0
    for val in vals:
        val = vals[count]
        if val != 0:
            rval *= val
            count+=1
    if count != 0:
        rval = pow(rval, 1.0/count)
    return(rval)

def geom_avg_mat(mat, coeffs = None):
    '''
    Computes the geometric average of the columns of a matrix.  Returns
    an np.array of dimension [nRowsOfMat], i.e. a vector.  
    
    :param mat: Must be an numpy.array of shape [nRows, nCols]
    :param coeffs:  If not None, it is a list like object with nColsOfMat elements.
    We multiply column 0 of mat by coeffs[0], column 1 of mat by coeffs[1], etc
    and then do the geometric average of the columns.  Essentially this weights the
    columns.
    '''
    """
    """
    size = mat.shape[0]
    rval = np.ones([size])
    for row in range(size):
        if np.any(coeffs):
            theRow = mat[row,:] * np.array(coeffs)
        else:
            theRow = mat[row,:]
        rval[row] = geom_avg(theRow)
    return(rval)

def bpriorities(mat, error = 1e-10):
    """
    Calculates priorities using Bill's method
    """
    size = mat.shape[0]
    vec = np.ones([size])
    diff = 1
    count=0
    while diff >= error and count < 100:
        nextv = geom_avg_mat(mat, vec)
        #nextv = nextv/max(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
        count+=1
    return(vec/sum(vec))

def gm_priorities(mat):
    '''
    Calculates the priorities using the geometric mean method
    :param mat: An numpy.array of dimension [size,size]
    '''
    rval = geom_avg_mat(mat)
    rval = rval / sum(rval)
    return(rval)

def harker_fix(mat):
    """
    Performs Harkers fix on the numpy matrix mat.  It returns a copy with the fix.
    The function does not change the matrix mat.
    :param mat: A numpy array
    :return: A copy of mat with Harker's fix applied to it
    """
    nrows = mat.shape[0]
    ncols = mat.shape[1]
    rval = mat.copy()
    for row in range(nrows):
        val = 1
        for col in range(ncols):
            if col != row and mat[row,col]==0:
                val+=1
        rval[row,row]=val
    return(rval)

def largest_eigen(mat, error = 1e-10, use_harker = False, initv = None):
    '''
    Calculates the largest eigen vector of a matrix
    
    :param mat: A square numpy array.
    :return: A numpy vector that is the normalized (sum to 1) largest eigenvector.
    '''
    if use_harker:
        mat = harker_fix(mat)
    size = mat.shape[0]
    if initv is not None:
        vec = initv
    else:
        vec = np.ones([size])
    diff = 1
    print(vec)
    while diff > error:
        nextv = np.matmul(mat, vec)
        nextv = nextv/sum(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
    return(vec)

def priority_error(pwmat, privec):
    rval = 0
    diffsum = 0
    count = 0
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if pwmat[i,j] != 0:
                diffsum += (pwmat[i,j] - privec[i]/privec[j])**2
                count += 1
    if count == 0:
        return 0
    else:
        return diffsum ** (1.0/2)
    
def ratio_priority_error(pwmat, privec):
    rval = 1
    diffsum = 0
    count = 0
    ratio = 1
    score = 0
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if (pwmat[i,j] >= 1) and (i != j):
                ratio = pwmat[i,j]/(privec[i]/privec[j])
                if ratio >= 1:
                    score = ratio - 1
                else:
                    score = 1/ratio - 1
                diffsum += score
                count += 1
                #print("ratio={} diffprod={}".format(ratio, diffprod))
    if count == 0:
        return 0
    else:
        return diffsum * (1.0/count)
    
def ratio_priority_error_prod(pwmat, privec):
    rval = 1
    diffprod = 1
    count = 0
    ratio = 1
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if (pwmat[i,j] >= 1) and (i != j):
                ratio = pwmat[i,j]/(privec[i]/privec[j])
                diffprod *= ratio
                count += 1
                print("ratio={} diffprod={}".format(ratio, diffprod))
    print(diffprod)
    if count == 0:
        return 0
    else:
        return diffprod ** (1.0/count)
    
def ratio_mat(pv):
    size = len(pv)
    rval = np.identity(n=size)
    for row in range(size):
        for col in range(size):
            rval[row,col]=pv[row]/pv[col]
    return rval

# First round of calculations

## The matrix and largest eigen

In [2]:
mat = np.array([[1, 2, 1/9], [1/2, 1, 2], [9, 1/2, 1]])
mat

array([[ 1.        ,  2.        ,  0.11111111],
       [ 0.5       ,  1.        ,  2.        ],
       [ 9.        ,  0.5       ,  1.        ]])

In [3]:
leigen = largest_eigen(mat, error=1e-15)
leigen

[ 1.  1.  1.]


array([ 0.18598961,  0.30706208,  0.50694832])

## Calculate the mismatch of the priority vector vs the pairwise matrix

In [4]:
priority_error(mat, leigen)

6.7801209182673734

## Check gradient of error function
It should be all zeroes at `leigen` if that is a minimimum.

In [5]:
my_error = lambda x: priority_error(mat, x)

In [6]:
from scipy.misc import derivative
from scipy.optimize import approx_fprime, minimize

In [7]:
eps = 1e-14
approx_fprime(leigen, my_error, eps)

array([ 11.36868377,   0.        ,  -4.26325641])

## Minimize priority_error(mat, *)
**It is not all zeroes, it looks like we can get a smaller error!**

In [8]:
rval = minimize(my_error, leigen, bounds=[(.01, 1), (.01, 1), (.01, 1)])

In [9]:
newpv = rval.x / sum(rval.x)
newpv

array([ 0.08367377,  0.23820359,  0.67812264])

In [10]:
# This is not the same, let's see them both together
leigen, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367377,  0.23820359,  0.67812264]))

In [11]:
# Let's look at the error of the eigen vs newpv
my_error(leigen), my_error(newpv)

(6.7801209182673734, 4.1537975084868064)

In [12]:
# Well that is surprising, approximately a 10% decline in error
# Let's consider the ratio matrix for both eigen and newpv
ratio_mat(leigen), None, ratio_mat(newpv)

(array([[ 1.        ,  0.60570686,  0.36688081],
        [ 1.65096362,  1.        ,  0.60570686],
        [ 2.72568089,  1.65096362,  1.        ]]),
 None,
 array([[ 1.        ,  0.35127   ,  0.12339033],
        [ 2.84681298,  1.        ,  0.35126919],
        [ 8.10436291,  2.84681957,  1.        ]]))

In [13]:
ratio_mat(leigen)

array([[ 1.        ,  0.60570686,  0.36688081],
       [ 1.65096362,  1.        ,  0.60570686],
       [ 2.72568089,  1.65096362,  1.        ]])

In [14]:
my_error(rval.x)

4.1537975084868064

## Try doing a similar calculation with my new priority calculation

In [15]:
bpv=bpriorities(mat)

In [16]:
leigen, bpv, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367377,  0.23820359,  0.67812264]))

Whoops, I forgot my new calculation agrees with the standard eigen for 3x3 matrices, let's just check for the transpose as well, for giggles

In [17]:
tbpv = bpriorities(mat.transpose())
ibpv = 1/bpv
ibpv = ibpv / sum(ibpv)
tbpv, ibpv

(array([ 0.50694832,  0.30706208,  0.18598961]),
 array([ 0.50694832,  0.30706208,  0.18598961]))

In [18]:
largest_eigen(mat.transpose())

[ 1.  1.  1.]


array([ 0.50694832,  0.30706208,  0.18598961])

## Trying a new idea for convergence

In [19]:
def bill_iter(mat, p):
    rval = p/sum(p)
    size = len(p)
    for i in range(size):
        c = 1
        for j in range(size):
            if mat[i,j] >= 1:
                err = mat[i,j]/(p[i]/p[j])
            else:
                err = mat[j,i]/(p[j]/p[i])
            c *= err
        c = c ** (1/(2*(size-1)))
        rval[i]*=c
    rval=rval/sum(rval)
    return rval

In [20]:
p1=bill_iter(mat, newpv)
show_vals = [p1]
for i in range(50):
    show_vals.append(bill_iter(mat, show_vals[-1]))
show_vals

[array([ 0.07469302,  0.31996727,  0.60533971]),
 array([ 0.07447814,  0.43346536,  0.4920565 ]),
 array([ 0.08331014,  0.5509117 ,  0.36577816]),
 array([ 0.1063368 ,  0.63085349,  0.26280971]),
 array([ 0.15685457,  0.64358611,  0.19955931]),
 array([ 0.25643088,  0.57251478,  0.17105434]),
 array([ 0.41191741,  0.42189191,  0.16619069]),
 array([ 0.56762398,  0.25202821,  0.18034781]),
 array([ 0.63755448,  0.13420867,  0.22823685]),
 array([ 0.57763266,  0.07374461,  0.34862272]),
 array([ 0.38421096,  0.0437701 ,  0.57201894]),
 array([ 0.16565286,  0.02721272,  0.80713443]),
 array([ 0.05194343,  0.02031331,  0.92774326]),
 array([ 0.01635334,  0.02346603,  0.96018063]),
 array([ 0.00696487,  0.04804052,  0.94499461]),
 array([ 0.00473132,  0.16104915,  0.83421954]),
 array([ 0.00417358,  0.53621238,  0.45961404]),
 array([ 0.0032528 ,  0.89437098,  0.10237622]),
 array([ 0.00376535,  0.97930676,  0.01692789]),
 array([ 0.01045061,  0.98530729,  0.0042421 ]),
 array([ 0.07008281,

Okay that was **STUPID**

## Trying ratio_priority_error instead

In [21]:
ratio_priority_error(mat, leigen), ratio_priority_error(mat, newpv)

(2.3019272488946267, 3.1659260204051973)

Okay, so the newpv is not as good on this measurement, let's try to minimize again

In [22]:
my_error_ratio = lambda x: ratio_priority_error(mat, x)

In [23]:
rval_ratio = minimize(my_error_ratio, [10,1,1], bounds=[(.01, 1), (.01, 1), (.01, 1)])

In [24]:
ratio_priority_error(mat, rval_ratio.x)

2.301927248894641

In [25]:
rval_ratio.x/sum(rval_ratio.x)

array([ 0.18598962,  0.30706205,  0.50694833])

In [26]:
leigen

array([ 0.18598961,  0.30706208,  0.50694832])

It seems like the ratio priority error minimizing vector is leigen in this case.  This is weird, but I think it might be an artifact of the largest eigen having the doppelganger property for 3x3 matrices.

Just for giggles, let's verify for this case that doppelganger error is the same as original

In [27]:
ratio_priority_error(mat, leigen), ratio_priority_error(mat.transpose(), 1/leigen)

(2.3019272488946267, 2.3019272488946263)

## Trying ratio priority error again, with a wildly inconsistent 4x4 instead

In [28]:
mat4 = np.array([
    [1, 8, 1/10, 1/10],
    [1/8, 1, 4, 1/10],
    [10 , 1/4, 1, 5],
    [10, 10, 1/5, 1]
])

In [29]:
leigen4 = largest_eigen(mat4)

[ 1.  1.  1.  1.]


In [30]:
my_error_ratio4 = lambda x: ratio_priority_error(mat4, x)

In [31]:
rval_ratio = minimize(my_error_ratio4, leigen4, bounds=[(.001, 1), (.001, 1), (.001, 1), (.001, 1)])

In [32]:
newpv4 = rval_ratio.x/sum(rval_ratio.x)

In [33]:
leigen4, newpv4

(array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183]),
 array([ 0.13517903,  0.12521498,  0.32696025,  0.41264574]))

In [34]:
ratio_priority_error(mat4, leigen4), ratio_priority_error(mat4, newpv4)

(4.9164739761337177, 4.7683649755766364)

Okay, we have an example where largest eigen disagrees with the new method.  Notice that they have different rankings of the alternatives as well

* largest_eigen: 4, 3, 1, 2
* new_idea     : 3, 4, 2, 1

In [35]:
bpv4 = bpriorities(mat4)
leigen4, newpv4, bpv4

(array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183]),
 array([ 0.13517903,  0.12521498,  0.32696025,  0.41264574]),
 array([ 0.10637135,  0.09457899,  0.37607953,  0.42297012]))

Notice that the `bpriorities()` are different than either the eigen or new method

# Exploring eigenvalue as error measure

In [36]:
def perr_eigen(mat, vec):
    size = np.sqrt(vec.dot(vec))
    newvec = vec/size
    nextv = np.matmul(mat, newvec)
    return np.sqrt(nextv.dot(nextv))

In [37]:
mat4

array([[  1.   ,   8.   ,   0.1  ,   0.1  ],
       [  0.125,   1.   ,   4.   ,   0.1  ],
       [ 10.   ,   0.25 ,   1.   ,   5.   ],
       [ 10.   ,  10.   ,   0.2  ,   1.   ]])

## Trying this new error

In [38]:
perr_eigen(mat4, leigen4)

10.155003605558392

In [39]:
perr_eigen(mat4, bpv4)

8.1672004292642466

In [40]:
perr_eigen(mat4, np.array([1,0,0,0]))

14.177997919311457

In [41]:
perr_eigen(mat4, np.array([0,1,0,0]))

12.847665157529597

In [42]:
np.matmul(mat4, np.array([0, 0, 1,0]))

array([ 0.1,  4. ,  1. ,  0.2])

In [43]:
perr_eigen(mat4, np.array([0,0,1,0]))

4.1291645644125161

In [44]:
perr_eigen(mat4, np.array([0,0,0,1]))

5.1009802979427397

In [45]:
perr_eigen(mat4, newpv4)

9.38880716167135

In [46]:
from scipy import linalg

In [47]:
linalg.eigvals(mat4)

array([ 10.15500361+0.j       ,  -2.06739864+7.0623511j,
        -2.06739864-7.0623511j,  -2.02020632+0.j       ])

In [48]:
a=mat4.dot(np.array([1, 0, 0, 0]))

In [49]:
a

array([  1.   ,   0.125,  10.   ,  10.   ])

In [50]:
a.dot(a)

201.015625

In [51]:
largest_eigen(mat4)

[ 1.  1.  1.  1.]


array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183])

In [52]:
largest_eigen(mat4, initv=np.array([0,0,0,1]))

[0 0 0 1]


array([ 0.14684422,  0.15937263,  0.35162132,  0.34216183])

In [53]:
mat4

array([[  1.   ,   8.   ,   0.1  ,   0.1  ],
       [  0.125,   1.   ,   4.   ,   0.1  ],
       [ 10.   ,   0.25 ,   1.   ,   5.   ],
       [ 10.   ,  10.   ,   0.2  ,   1.   ]])

In [54]:
info = linalg.eig(mat4)
evals = info[0]
evecs = info[1]

In [55]:
mat4.dot(evecs[0])/evecs[0]

array([  8.56143852-7.29751524j,   1.10709688+3.98461127j,
         2.33779808-1.54044531j, -12.06917842+6.00201815j])

In [56]:
evecs[0]

array([ 0.27378251+0.j        ,  0.26067693-0.25290279j,
        0.26067693+0.25290279j, -0.41293566+0.j        ])

In [57]:
evals

array([ 10.15500361+0.j       ,  -2.06739864+7.0623511j,
        -2.06739864-7.0623511j,  -2.02020632+0.j       ])

In [58]:
?linalg.eig

In [59]:
evecs

array([[ 0.27378251+0.j        ,  0.26067693-0.25290279j,
         0.26067693+0.25290279j, -0.41293566+0.j        ],
       [ 0.29714100+0.j        ,  0.13327516+0.33327178j,
         0.13327516-0.33327178j,  0.14625482+0.j        ],
       [ 0.65557751+0.j        , -0.69624664+0.j        ,
        -0.69624664-0.j        , -0.11979874+0.j        ],
       [ 0.63794083+0.j        , -0.10088442-0.49428566j,
        -0.10088442+0.49428566j,  0.89092196+0.j        ]])

# Limit matrix calculations

In [62]:
mat = np.array([
    [0.3, 0.2, 0.4],
    [0.1, 0.5, 0.5],
    [0.6, 0.3, 0.1]
])

In [63]:
int(np.ceil(np.log2(1.5)))

1

In [81]:
from copy import deepcopy
def _mat_pow2(mat, power):
    n = int(np.ceil(np.log2(power)))
    last = deepcopy(mat)
    count = 0
    nextm = deepcopy(mat)
    for i in range(n):
        np.matmul(last, last, nextm)
        tmp = last
        last = nextm
        nextm = tmp
    return nextm

In [79]:
a = np.matmul(mat, mat)
np.matmul(a,a)

array([[ 0.2991,  0.2936,  0.2902],
       [ 0.3844,  0.3848,  0.3776],
       [ 0.3165,  0.3216,  0.3322]])

In [66]:
_mat_pow2(mat, 5)

3


array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [67]:
def calculus(mat, error=1e-10, max_iters=1000):
    size = len(mat)
    diff = 0.0
    start_pow = 1
    start = _mat_pow2(mat, start_pow)
    tmp1 = deepcopy(mat)
    tmp2 = deepcopy(mat)
    tmp3 = deepcopy(mat)
    #Now we need matrices to store the intermediate results
    pows = [start]
    for i in range(size-1):
        #Add next one
        pows.append(np.matmul(mat, pows[-1]))
        diff = normalize_cols_dist(pows[-1], pows[-2], tmp1, tmp2, tmp3)
        #print(diff)
        if diff < error:
            #Already converged, done
            return pows[-1]
    #print(pows)
    for count in range(max_iters):
        nextp = pows[0]
        np.matmul(pows[-1], mat, nextp)
        #print(pows)
        for i in range(len(pows)-1):
            pows[i]=pows[i+1]
        pows[-1]=nextp
        #print(pows)
        #Check convergence
        for i in range(len(pows)-1):
            diff = normalize_cols_dist(pows[i], nextp, tmp1, tmp2, tmp3)
            #print(pows[i])
            #print(nextp)
            #print(diff)
            if diff < error:
                return nextp / nextp.sum(axis=0)
            

def normalize_cols_dist(mat1, mat2, tmp1, tmp2, tmp3):
    np.divide(mat1, mat1.max(axis=0), tmp1)
    np.divide(mat2, mat2.max(axis=0), tmp2)
    np.subtract(tmp1, tmp2, tmp3)
    np.absolute(tmp3, tmp3)
    return np.max(tmp3)

In [68]:
calculus(mat, error=1e-16)

0


array([[ 0.29411765,  0.29411765,  0.29411765],
       [ 0.38235294,  0.38235294,  0.38235294],
       [ 0.32352941,  0.32352941,  0.32352941]])

In [69]:
calculus(mat, error=1e-8)

0


array([[ 0.29411765,  0.29411765,  0.29411765],
       [ 0.38235294,  0.38235294,  0.38235294],
       [ 0.32352941,  0.32352941,  0.32352941]])

In [70]:
mat10 = np.random.random_sample((10,10))
mat10 = mat10 / mat10.sum(axis=0)
mat10

array([[ 0.0164158 ,  0.16647807,  0.07340726,  0.13332282,  0.03430357,
         0.14588821,  0.02403746,  0.13667474,  0.08926599,  0.10678425],
       [ 0.15992233,  0.16211991,  0.10241658,  0.03875266,  0.0707179 ,
         0.08979368,  0.18986215,  0.10343115,  0.049502  ,  0.08835696],
       [ 0.09545074,  0.18201421,  0.05079827,  0.10428844,  0.20243491,
         0.14704495,  0.10964769,  0.06247938,  0.10419825,  0.13905349],
       [ 0.15400157,  0.01880858,  0.12973263,  0.15914768,  0.0095083 ,
         0.12024423,  0.00272151,  0.12967683,  0.18089913,  0.04335349],
       [ 0.07275198,  0.07580903,  0.14829212,  0.17720603,  0.08933466,
         0.04296714,  0.1626025 ,  0.08597098,  0.071895  ,  0.09219052],
       [ 0.03840237,  0.12245178,  0.09527248,  0.09295918,  0.05186789,
         0.06844837,  0.10357168,  0.0534983 ,  0.06222093,  0.13765593],
       [ 0.14883741,  0.11985546,  0.15087366,  0.13801914,  0.32903213,
         0.0784559 ,  0.14003364,  0.07845363

In [71]:
calculus(mat10)[:,0]

0


array([ 0.08856738,  0.11214952,  0.11907798,  0.08711528,  0.10658461,
        0.08376521,  0.14960254,  0.09974984,  0.07979819,  0.07358946])

In [86]:
mat2 = np.array([[1,1], [1,1]])
_mat_pow2(mat2, 5)

3


array([[8, 8],
       [8, 8]])